In [25]:
import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time, logging
import mhcflurry.data
import mhcflurry.imputation
import fancyimpute

import sklearn.metrics
import sklearn.cross_validation
from collections import defaultdict

In [26]:
min_peptides_to_consider_allele = 50
max_ic50 = 50000
data_dir = "../data/"

In [27]:
all_train_data = mhcflurry.data.load_allele_datasets(
    data_dir + "bdata.2009.mhci.public.1.txt",
    use_multiple_peptide_lengths=True)

In [28]:
alleles = [
    "HLA-A0201",
    # "HLA-A0301",
    # "HLA-A0203",
    # "HLA-A2602",
    # "HLA-A2603",
    # 'HLA-B7301',
]
#alleles = alleles[:1] + alleles[-1:]
#alleles = [allele for allele in all_train_data if len(all_train_data[allele].Y) >= min_peptides_to_consider_allele]

In [29]:
all_train_data[alleles[0]].weights.std()

0.36749263596306014

In [30]:
set(len(x) for x in all_train_data[alleles[0]].original_peptides)

{8, 9, 10, 11, 12, 13, 14, 15}

In [31]:
set(len(x) for x in all_train_data[alleles[0]].peptides)

{9}

In [32]:
#train_data = dict((allele, data)
#                  for (allele, data) in all_train_data.items()
#                  if len(data.Y) >= min_peptides_to_consider_allele)
train_data = dict((allele, all_train_data[allele]) for allele in alleles)
print("Training data: %d / %d alleles" % (len(train_data), len(all_train_data)))

#test_data = mhcflurry.data.load_allele_datasets("/Users/tim/sinai/git/mhcflurry/bdata.2013.mhci.public.blind.1.txt")


Training data: 1 / 106 alleles


In [33]:
def log_to_ic50(log_value):
        """
        Convert neural network output to IC50 values between 0.0 and
        self.max_ic50 (typically 5000, 20000 or 50000)
        """
        return max_ic50 ** (1.0 - log_value)

def make_scores(y, y_pred, weights=None, sample_weight=None, threshold_nm=500):
    ic50_y = log_to_ic50(y)
    ic50_y_pred = log_to_ic50(y_pred) 
    return dict(
        auc=sklearn.metrics.roc_auc_score(ic50_y <= threshold_nm, y_pred, sample_weight=sample_weight),
        f1=sklearn.metrics.f1_score(ic50_y <= threshold_nm, ic50_y_pred <= threshold_nm, sample_weight=sample_weight),
        tau=scipy.stats.kendalltau(y_pred, y)[0],
    )    

def mean_with_std(grouped_column, decimals=3):
    pattern = "%%0.%df" % decimals
    return pandas.Series([
        (pattern + " +/ " + pattern) % (m, s) if not pandas.isnull(s) else pattern % m
        for (m, s) in zip(grouped_column.mean(), grouped_column.std())
    ], index = grouped_column.mean().index)

def allele_data_to_df(data):
    d = data._asdict()
    d["X_index"] = [x for x in d["X_index"]]
    d["X_binary"] = [x for x in d["X_binary"]]
    df = pandas.DataFrame(d).set_index('peptides')
    return df

def make_2d_array(thing):
    return numpy.array([list(x) for x in thing])

def df_to_allele_data(df):
    d = dict((col, df[col].values) for col in df)
    d["X_index"] = make_2d_array(d["X_index"])
    (d["max_ic50"],) = list(df.max_ic50.unique())
    return mhcflurry.data.AlleleData(peptides = df.index.values, **d)

def get_shuffled_fields(allele_data):
    """
    Parameters
    ----------
    allele_data : mhcflurry.data.AlleleData
    
    Returns shuffled array fields
    """
    original_peptides = data.original_peptides
    X = data.X_index 
    Y = data.Y
    weights = data.weights
    
    # shuffle all the  samples!
    shuffle_indices = np.arange(len(X))
    np.random.shuffle(shuffle_indices)
    
    X = X[shuffle_indices]
    Y = Y[shuffle_indices]
    weights = weights[shuffle_indices]
    original_peptides = original_peptides[shuffle_indices]
    
    return X, Y, weights, original_peptides

def collapse_9mer_affinities(Y_9mer_true, Y_9mer_pred, original_peptides):
    """
    Parameters
    ----------
    Y_9mer_true : np.array of float
        True regression target values for 9mers extracted from longer/shorter peptides
    
    Y_9mer_pred : np.array of float
        Predicted values for 9mers
    
    original_peptides : np.array of str
        Original peptides of varying length that 9mers were extracted from.
    """
    # collapse multiple 9mer predictions and measured values into 
    # smaller set of predictions for peptides of varying lengths
    peptide_to_true_affinity_dict = defaultdict(list)
    peptide_to_predicted_affinity_dict = defaultdict(list)
    for i, p in enumerate(original_peptides):
        peptide_to_true_affinity_dict[p].append(Y_9mer_true[i])
        peptide_to_predicted_affinity_dict[p].append(Y_9mer_pred[i])

    unique_peptides = list(sorted(set(peptide_to_predicted_affinity_dict.keys())))
    print("-- # unique peptides = %d" % (len(unique_peptides),))
    Y_true = np.array([
            np.mean(peptide_to_true_affinity_dict[p]) for p in unique_peptides ])
    Y_pred = np.array([
            np.mean(peptide_to_predicted_affinity_dict[p]) for p in unique_peptides])
    return Y_true, Y_pred

In [45]:
dropout_probabilities = [0.0, 0.5]

embedding_output_dims = [8, 32]
#embedding_output_dims = [4, 32]

#layer_sizes = [[4], [8], [16], [64], [128]]
layer_sizes_list = [[4], [64]]

activations = ["tanh"]

models_params_list = []
for dropout_probability in dropout_probabilities:
    for embedding_output_dim in embedding_output_dims:
        for layer_sizes in layer_sizes_list:
            for activation in activations:
                models_params_list.append(dict(
                    dropout_probability=dropout_probability,  
                    embedding_output_dim=embedding_output_dim,
                    layer_sizes=layer_sizes,
                    activation=activation))

print("%d models" % len(models_params_list))
models_params_explored = set.union(*[set(x) for x in models_params_list])
models_params_explored


8 models


{'activation', 'dropout_probability', 'embedding_output_dim', 'layer_sizes'}

In [46]:
import sys


cv_df = defaultdict(list)
start = time.time()

for (allele, data) in train_data.items():
    print("Allele: %s" % allele)
    # data_df = allele_data_to_df(data)
    X, Y, weights, original_peptides = get_shuffled_fields(data)
            
    cv = sklearn.cross_validation.LabelKFold(original_peptides, n_folds = 3)
    for (fold_num, (train_indices, test_indices)) in enumerate(cv):
        print("-- fold #%d/3" % (fold_num + 1,))
        X_cv_train = X[train_indices]
        X_cv_test = X[test_indices]
        
        Y_cv_train = Y[train_indices]
        Y_cv_test = Y[test_indices]
        
        weights_cv_train = weights[train_indices]
        weights_cv_test = weights[test_indices]
        
        original_peptides_train = original_peptides[train_indices]
        original_peptides_test = original_peptides[test_indices]
        impute = False
        
        np.random.shuffle(models_params_list)
        for (i, model_params) in enumerate(models_params_list):
            print("%10s fold %3d [%3d / %3d] train_size=%d test_size=%d impute=%s model=%s" %
                  (allele, fold_num, i, len(models_params_list), len(train_indices), len(test_indices), impute, model_params))
            sys.stdout.flush()
            predictor = mhcflurry.Class1BindingPredictor.from_hyperparameters(
                max_ic50=max_ic50,
                **model_params)

            fit_time = -time.time()
            predictor.fit(
                X_cv_train,
                Y_cv_train,
                sample_weights=weights_cv_train,
                verbose=False,
                n_training_epochs=250)
            fit_time += time.time()
    
            Y_cv_train_9mer_predictions = predictor.predict(X_cv_train)
            Y_cv_test_9mer_predictions = predictor.predict(X_cv_test)
            
            Y_train_true, Y_train_pred = collapse_9mer_affinities(
                Y_9mer_true=Y_cv_train,
                Y_9mer_pred=Y_cv_train_9mer_predictions,
                original_peptides=original_peptides_train)
            
            Y_test_true, Y_test_pred = collapse_9mer_affinities(
                Y_9mer_true=Y_cv_test,
                Y_9mer_pred=Y_cv_test_9mer_predictions,
                original_peptides=original_peptides_test)
            
            cv_df["allele"].append(allele)
            cv_df["allele_size"].append(len(Y))
            cv_df["train_size"].append(len(Y_cv_train))
            cv_df["model_params"].append(model_params)
            cv_df["fit_time"].append(fit_time)

            for (param, param_value) in model_params.items():
                cv_df[param].append(param_value)
            
            for (key, value) in make_scores(Y_train_true, Y_train_pred).items():
                cv_df["train_%s" % key].append(value)
                print("train %s: %f" % (key, value))
            
            for (key, value) in make_scores(
                    Y_test_true, 
                    Y_test_pred).items():
                cv_df["test_%s" % key].append(value)
                print("test %s: %f" % (key, value))


cv_df = pandas.DataFrame(cv_df)
cv_df["layer0_size"] = [x[0] for x in cv_df.layer_sizes]
print(time.time() - start)
cv_df

Allele: HLA-A0201
-- fold #1/3
 HLA-A0201 fold   0 [  0 /   8] train_size=21917 test_size=10959 impute=False model={'activation': 'tanh', 'embedding_output_dim': 8, 'dropout_probability': 0.0, 'layer_sizes': [4]}
-- # unique peptides = 6377
-- # unique peptides = 3188
train tau: 0.635373
train auc: 0.961440
train f1: 0.857345
test tau: 0.612421
test auc: 0.947852
test f1: 0.812253
 HLA-A0201 fold   0 [  1 /   8] train_size=21917 test_size=10959 impute=False model={'activation': 'tanh', 'embedding_output_dim': 8, 'dropout_probability': 0.5, 'layer_sizes': [4]}
-- # unique peptides = 6377
-- # unique peptides = 3188
train tau: 0.595930
train auc: 0.941433
train f1: 0.816062
test tau: 0.593591
test auc: 0.934945
test f1: 0.794962
 HLA-A0201 fold   0 [  2 /   8] train_size=21917 test_size=10959 impute=False model={'activation': 'tanh', 'embedding_output_dim': 32, 'dropout_probability': 0.0, 'layer_sizes': [64]}
-- # unique peptides = 6377
-- # unique peptides = 3188
train tau: 0.853621
tra

,activation,allele,allele_size,dropout_probability,embedding_output_dim,fit_time,layer_sizes,model_params,test_auc,test_f1,test_tau,train_auc,train_f1,train_size,train_tau,layer0_size
0,tanh,HLA-A0201,32876,0.0,8,84.926343,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.947852,0.812253,0.612421,0.961440,0.857345,21917,0.635373,4
1,tanh,HLA-A0201,32876,0.5,8,96.526969,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.934945,0.794962,0.593591,0.941433,0.816062,21917,0.595930,4
2,tanh,HLA-A0201,32876,0.0,32,87.635577,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.893769,0.727977,0.504335,0.998382,0.969092,21917,0.853621,64
3,tanh,HLA-A0201,32876,0.5,32,99.237046,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.945726,0.810010,0.609418,0.951109,0.828886,21917,0.612068,4
4,tanh,HLA-A0201,32876,0.0,32,87.175426,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.941599,0.793750,0.592833,0.968903,0.851287,21917,0.649106,4
5,tanh,HLA-A0201,32876,0.5,8,98.160665,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.937999,0.786494,0.596894,0.943413,0.809371,21917,0.598930,64
6,tanh,HLA-A0201,32876,0.0,8,86.410095,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.897357,0.732064,0.522877,0.995456,0.955315,21917,0.775955,64
7,tanh,HLA-A0201,32876,0.5,32,100.420329,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.947580,0.822068,0.613136,0.952590,0.843105,21917,0.617173,64
8,tanh,HLA-A0201,32876,0.0,32,86.968521,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.951407,0.826462,0.599771,0.966108,0.860940,21917,0.658287,4
9,tanh,HLA-A0201,32876,0.5,32,98.788730,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.949710,0.841608,0.603994,0.949856,0.829387,21917,0.617840,4


In [51]:
cv_df.shape

(24, 17)

In [78]:
cv_df_str = cv_df.copy()
print(cv_df_str.columns)
del cv_df_str['model_params']
del cv_df_str['fit_time']

for col in ["layer_sizes"]:
    cv_df_str[col] = [str(x) for x in cv_df_str[col]]
summary = cv_df_str.groupby(list(cv_df_str.columns[:6])).mean() #.reset_index()
summary.sort("combined", ascending=False, inplace=True)
summary.to_csv("../data/cv_hla0201_summary.csv")

Index([u'activation', u'allele', u'allele_size', u'dropout_probability',
       u'embedding_output_dim', u'fit_time', u'layer_sizes', u'model_params',
       u'test_auc', u'test_f1', u'test_tau', u'train_auc', u'train_f1',
       u'train_size', u'train_tau', u'layer0_size', u'combined'],
      dtype='object')


/home/tim/anaconda2/envs/standard-2.7/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [49]:
cv_df["combined"] = cv_df.test_auc + cv_df.test_f1 + cv_df.test_tau

In [50]:
cv_df.sort("combined", ascending=False, inplace=True)
cv_df

/home/tim/anaconda2/envs/standard-2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,activation,allele,allele_size,dropout_probability,embedding_output_dim,fit_time,layer_sizes,model_params,test_auc,test_f1,test_tau,train_auc,train_f1,train_size,train_tau,layer0_size,combined
11,tanh,HLA-A0201,32876,0.0,8,84.933377,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.953694,0.836469,0.607306,0.959798,0.844069,21917,0.641526,4,2.397469
13,tanh,HLA-A0201,32876,0.5,32,100.035493,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.951017,0.837929,0.606459,0.951691,0.830517,21917,0.622023,64,2.395405
9,tanh,HLA-A0201,32876,0.5,32,98.788730,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.949710,0.841608,0.603994,0.949856,0.829387,21917,0.617840,4,2.395312
17,tanh,HLA-A0201,32876,0.5,32,100.653926,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.947980,0.827857,0.613741,0.954082,0.832507,21918,0.619337,64,2.389578
16,tanh,HLA-A0201,32876,0.0,8,85.211551,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.947806,0.824917,0.612441,0.962871,0.857006,21918,0.636318,4,2.385164
7,tanh,HLA-A0201,32876,0.5,32,100.420329,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.947580,0.822068,0.613136,0.952590,0.843105,21917,0.617173,64,2.382784
8,tanh,HLA-A0201,32876,0.0,32,86.968521,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.951407,0.826462,0.599771,0.966108,0.860940,21917,0.658287,4,2.377641
19,tanh,HLA-A0201,32876,0.5,32,99.454345,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.945700,0.824331,0.607017,0.951755,0.833457,21918,0.613908,4,2.377048
0,tanh,HLA-A0201,32876,0.0,8,84.926343,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.947852,0.812253,0.612421,0.961440,0.857345,21917,0.635373,4,2.372526
3,tanh,HLA-A0201,32876,0.5,32,99.237046,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.945726,0.810010,0.609418,0.951109,0.828886,21917,0.612068,4,2.365154


In [47]:
cv_df = pandas.DataFrame(cv_df)
cv_df["layer0_size"] = [x[0] for x in cv_df.layer_sizes]
cv_df

,activation,allele,allele_size,dropout_probability,embedding_output_dim,fit_time,layer_sizes,model_params,test_auc,test_f1,test_tau,train_auc,train_f1,train_size,train_tau,layer0_size
0,tanh,HLA-A0201,32876,0.0,8,84.926343,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.947852,0.812253,0.612421,0.961440,0.857345,21917,0.635373,4
1,tanh,HLA-A0201,32876,0.5,8,96.526969,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.934945,0.794962,0.593591,0.941433,0.816062,21917,0.595930,4
2,tanh,HLA-A0201,32876,0.0,32,87.635577,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.893769,0.727977,0.504335,0.998382,0.969092,21917,0.853621,64
3,tanh,HLA-A0201,32876,0.5,32,99.237046,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.945726,0.810010,0.609418,0.951109,0.828886,21917,0.612068,4
4,tanh,HLA-A0201,32876,0.0,32,87.175426,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.941599,0.793750,0.592833,0.968903,0.851287,21917,0.649106,4
5,tanh,HLA-A0201,32876,0.5,8,98.160665,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.937999,0.786494,0.596894,0.943413,0.809371,21917,0.598930,64
6,tanh,HLA-A0201,32876,0.0,8,86.410095,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.897357,0.732064,0.522877,0.995456,0.955315,21917,0.775955,64
7,tanh,HLA-A0201,32876,0.5,32,100.420329,[64],"{u'activation': u'tanh', u'embedding_output_di...",0.947580,0.822068,0.613136,0.952590,0.843105,21917,0.617173,64
8,tanh,HLA-A0201,32876,0.0,32,86.968521,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.951407,0.826462,0.599771,0.966108,0.860940,21917,0.658287,4
9,tanh,HLA-A0201,32876,0.5,32,98.788730,[4],"{u'activation': u'tanh', u'embedding_output_di...",0.949710,0.841608,0.603994,0.949856,0.829387,21917,0.617840,4


In [48]:
cv_df.to_csv("cv5.csv")

In [ ]:
group_columns = ["allele", "allele_size", "impute"]
group_columns.extend(models_params_explored)
group_columns.append("layer0_size")
group_columns.remove("layer_sizes")
print(mean_with_std(cv_df.groupby(group_columns).test_auc)) #.sort(inplace=False, ascending=False)



In [ ]:
def best_by(score):
    means = cv_df.groupby(group_columns)[score].mean().reset_index()
    max_rows = []
    for allele in means.allele.unique():
        max_rows.append(means.ix[means.allele == allele][score].argmax())
    return means.ix[max_rows]

In [ ]:
best_by('test_auc')


In [ ]:
best_by('test_tau')

In [ ]:
best_by('test_f1')